In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.linear_model import LinearRegression
from datetime import timedelta
import warnings

# Suppress warnings for a cleaner output
warnings.filterwarnings("ignore")

# --- Configuration ---
# You can adjust these parameters to fit your specific needs.
DATA_PERIOD_DAYS = 365 * 2 # How many days of historical data to generate.
FORECAST_PERIOD_DAYS = 30  # How many days into the future to forecast.
ALERT_TO_CASE_LAG_DAYS = 1 # The typical delay (in days) for an alert to become a case.
BASE_CONVERSION_RATE = 0.05 # ~5% of alerts become cases.
WEEKLY_SEASONALITY = True   # Set to True if you expect weekly patterns (e.g., fewer alerts on weekends).

# --- 1. Generate Realistic Sample Data ---
# In a real-world scenario, you would replace this section with your own data loading logic.
# For example: df = pd.read_csv('your_data.csv', parse_dates=['date_column'])
def generate_sample_data():
    """
    Creates a sample DataFrame with 'date', 'alerts', and 'cases' columns.
    This simulates a year of historical data with trends and seasonality.
    """
    print(f"Generating {DATA_PERIOD_DAYS} days of sample data...")
    
    # Create a date range
    start_date = pd.to_datetime('2023-01-01')
    dates = pd.date_range(start_date, periods=DATA_PERIOD_DAYS, freq='D')
    
    # Generate alerts data
    # Base alerts + some noise + a slight upward trend + weekly seasonality
    base_alerts = 200
    noise = np.random.randint(-50, 50, size=DATA_PERIOD_DAYS)
    trend = np.arange(DATA_PERIOD_DAYS) * 0.2
    
    alerts = base_alerts + noise + trend
    
    # Add weekly seasonality (e.g., fewer alerts on weekends)
    if WEEKLY_SEASONALITY:
        day_of_week_effect = [0, 0, 0, 0, -20, -80, -90] # Mon=0, Tue=1... Sat=5, Sun=6
        seasonality = [day_of_week_effect[d.weekday()] for d in dates]
        alerts += seasonality

    # Ensure alerts are non-negative
    alerts = np.maximum(0, alerts).astype(int)

    # Generate cases data based on lagged alerts and conversion rate
    # Cases on a given day are a percentage of alerts from 'LAG_DAYS' ago + some noise
    base_cases = (pd.Series(alerts).shift(ALERT_TO_CASE_LAG_DAYS) * BASE_CONVERSION_RATE).fillna(0)
    case_noise = np.random.randint(-2, 3, size=DATA_PERIOD_DAYS)
    cases = np.maximum(0, base_cases + case_noise).astype(int)
    
    # Create DataFrame
    df = pd.DataFrame({'date': dates, 'alerts': alerts, 'cases': cases})
    df.set_index('date', inplace=True)
    
    print("Sample data generation complete.\n")
    return df

# --- 2. Forecast Future Alerts using SARIMA ---
def forecast_future_alerts(df_historical):
    """
    Uses a SARIMA model to forecast the number of alerts for the next N days.
    
    Args:
        df_historical (pd.DataFrame): DataFrame with historical alerts data.
        
    Returns:
        pd.Series: A series of forecasted alert counts, indexed by future dates.
    """
    print(f"Training SARIMA model to forecast alerts for the next {FORECAST_PERIOD_DAYS} days...")
    
    # SARIMA model parameters. These may need tuning for your specific dataset.
    # (p,d,q) are the non-seasonal parameters
    # (P,D,Q,m) are the seasonal parameters. m=7 for weekly seasonality.
    order = (1, 1, 1)
    seasonal_order = (1, 1, 0, 7) if WEEKLY_SEASONALITY else (0, 0, 0, 0)
    
    # Initialize and fit the SARIMA model
    sarima_model = SARIMAX(df_historical['alerts'],
                           order=order,
                           seasonal_order=seasonal_order,
                           enforce_stationarity=False,
                           enforce_invertibility=False)
    
    model_fit = sarima_model.fit(disp=False)
    
    # Get the forecast for the specified number of future days
    forecast_result = model_fit.get_forecast(steps=FORECAST_PERIOD_DAYS)
    
    # Extract the predicted mean values and ensure they are non-negative
    forecasted_alerts = forecast_result.predicted_mean
    forecasted_alerts = forecasted_alerts.apply(lambda x: max(0, x)).astype(int)
    
    print("Alert forecasting complete.\n")
    return forecasted_alerts

# --- 3. Model Alert-to-Case Conversion and Predict Future Cases ---
def predict_future_cases(df_historical, forecasted_alerts):
    """
    Models the relationship between lagged alerts and cases, then uses this model
    to predict future cases based on forecasted alerts.
    
    Args:
        df_historical (pd.DataFrame): DataFrame with historical data.
        forecasted_alerts (pd.Series): Series of forecasted alerts.
        
    Returns:
        pd.Series: A series of forecasted case counts, indexed by future dates.
    """
    print("Modeling alert-to-case conversion rate...")
    
    # Feature Engineering: Create a lagged alerts column to correlate with cases
    df_prepared = df_historical.copy()
    df_prepared['alerts_lagged'] = df_prepared['alerts'].shift(ALERT_TO_CASE_LAG_DAYS)
    
    # Drop rows with NaN values resulting from the shift operation
    df_prepared.dropna(inplace=True)
    
    # Prepare data for Scikit-Learn Linear Regression model
    X = df_prepared[['alerts_lagged']] # Feature
    y = df_prepared['cases']           # Target
    
    # Train the linear regression model
    lr_model = LinearRegression()
    lr_model.fit(X, y)
    
    # The learned conversion rate is the model's coefficient
    learned_rate = lr_model.coef_[0]
    print(f"Learned Conversion Rate: {learned_rate:.2%}")
    print("Predicting future cases based on forecasted alerts...")

    # Predict future cases using the trained model and the forecasted alerts
    # We need to reshape the forecasted_alerts series for the model's predict method
    future_alerts_reshaped = forecasted_alerts.values.reshape(-1, 1)
    predicted_cases = lr_model.predict(future_alerts_reshaped)
    predicted_cases = np.maximum(0, predicted_cases).astype(int) # Ensure non-negative
    
    # Create a pandas Series for the predicted cases with the correct future dates
    future_cases_series = pd.Series(predicted_cases, index=forecasted_alerts.index)

    print("Case prediction complete.\n")
    return future_cases_series
    
# --- 4. Visualize the Results ---
def plot_forecast(historical_cases, forecasted_cases):
    """
    Plots the historical cases and the forecasted cases on a single graph.
    """
    print("Generating forecast plot...")
    plt.style.use('seaborn-v0_8-whitegrid')
    fig, ax = plt.subplots(figsize=(15, 7))
    
    # Plot historical data
    ax.plot(historical_cases.index, historical_cases, label='Historical Daily Cases', color='royalblue')
    
    # Plot forecasted data
    ax.plot(forecasted_cases.index, forecasted_cases, label='Forecasted Daily Cases', color='darkorange', linestyle='--')
    
    # Add a vertical line to separate historical from forecasted data
    last_historical_date = historical_cases.index[-1]
    ax.axvline(x=last_historical_date, color='crimson', linestyle=':', linewidth=2, label='Forecast Start')
    
    # Formatting
    ax.set_title(f'Forecast of Daily Case Creation for the Next {FORECAST_PERIOD_DAYS} Days', fontsize=16)
    ax.set_xlabel('Date', fontsize=12)
    ax.set_ylabel('Number of Cases', fontsize=12)
    ax.legend(fontsize=10)
    ax.grid(True, which='both', linestyle='--', linewidth=0.5)
    
    # Improve date formatting on the x-axis
    fig.autofmt_xdate()
    
    plt.tight_layout()
    plt.show()

# --- Main Execution ---
if __name__ == '__main__':
    # 1. Get data
    historical_df = generate_sample_data()
    
    # 2. Forecast the driver variable (alerts)
    future_alerts_forecast = forecast_future_alerts(historical_df)
    
    # 3. Use forecasted alerts to predict the target variable (cases)
    future_cases_forecast = predict_future_cases(historical_df, future_alerts_forecast)
    
    # 4. Display the results
    print("--- Forecast Results ---")
    forecast_df = pd.DataFrame({
        'Forecasted_Alerts': future_alerts_forecast,
        'Forecasted_Cases': future_cases_forecast
    })
    print(forecast_df)
    print("\n------------------------\n")
    
    # 5. Plot the forecast
    plot_forecast(historical_df['cases'], future_cases_forecast)



Generating 730 days of sample data...
Sample data generation complete.

Training SARIMA model to forecast alerts for the next 30 days...
Alert forecasting complete.

Modeling alert-to-case conversion rate...
Learned Conversion Rate: 4.98%
Predicting future cases based on forecasted alerts...
Case prediction complete.

--- Forecast Results ---
            Forecasted_Alerts  Forecasted_Cases
2024-12-31                370                17
2025-01-01                351                16
2025-01-02                388                18
2025-01-03                337                16
2025-01-04                258                12
2025-01-05                243                11
2025-01-06                351                16
2025-01-07                380                18
2025-01-08                355                17
2025-01-09                386                18
2025-01-10                328                15
2025-01-11                256                12
2025-01-12                265  

OSError: 'seaborn-v0_8-whitegrid' not found in the style library and input is not a valid URL or path; see `style.available` for list of available styles

In [3]:
pip install sklearn

    ERROR: Command errored out with exit status 1:
     command: /Users/joemcdougall/opt/anaconda3/bin/python -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/9l/3n2d6lxj18g2zlvdpv__ptm00000gn/T/pip-install-hy0kfrxy/sklearn/setup.py'"'"'; __file__='"'"'/private/var/folders/9l/3n2d6lxj18g2zlvdpv__ptm00000gn/T/pip-install-hy0kfrxy/sklearn/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /private/var/folders/9l/3n2d6lxj18g2zlvdpv__ptm00000gn/T/pip-pip-egg-info-2090lmaq
         cwd: /private/var/folders/9l/3n2d6lxj18g2zlvdpv__ptm00000gn/T/pip-install-hy0kfrxy/sklearn/
    Complete output (15 lines):
    The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
    rather than 'sklearn' for pip commands.
    
    Here is how to fix this error in the main use cases:
    - use 'pip install scikit-learn' rather than '

In [6]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [10]:
def detect_outliers_mad(data, threshold=3.5):
    """
    Identifies outliers in a dataset using the Median Absolute Deviation (MAD) method.

    Args:
        data (array-like): A list or numpy array of numerical data
                           (e.g., transaction amounts).
        threshold (float): The modified Z-score threshold to classify outliers.
                           Points with abs(modified Z-score) > threshold are
                           considered outliers. Common values are 3.0 or 3.5.
                           Defaults to 3.5.

    Returns:
        tuple: A tuple containing:
            - outliers (np.ndarray): An array of the data points identified as outliers.
            - median (float): The median of the input data.
            - mad (float): The Median Absolute Deviation of the input data.
            - modified_z_scores (np.ndarray or None): The calculated modified Z-scores
                                                      for each data point, or None if MAD is zero.
    """
# Ensure input is a numpy array for efficient calculations
    data = np.asarray(data)

    # Handle empty data gracefully
    if data.size == 0:
        return np.array([]), np.nan, np.nan, np.array([])

    # 1. Calculate the median of the data
    median = np.median(data)

    # 2. Calculate the absolute deviations from the median
    # Use np.abs for element-wise absolute value
    abs_deviations = np.abs(data - median)

    # 3. Calculate the Median Absolute Deviation (MAD)
    mad = np.median(abs_deviations)

    # 4. Handle the edge case where MAD is zero (e.g., >50% data points are identical)
    if mad == 0:
        print("Warning: MAD is zero. Cannot calculate modified Z-scores.")
        # Option 1: Return no outliers based on Z-score method
        # return np.array([]), median, mad, None
        # Option 2: Consider any point not equal to the median as an outlier
        outliers = data[data != median]
        return outliers, median, mad, None


    # 5. Calculate the Modified Z-scores
    # The constant 0.6745 scales MAD to be comparable to standard deviation
    # for normally distributed data.
    modified_z_scores = 0.6745 * abs_deviations / mad

    # 6. Identify outliers: points where the absolute modified Z-score > threshold
    outlier_mask = np.abs(modified_z_scores) > threshold
    outliers = data[outlier_mask]

    return outliers, median, mad, modified_z_scores

In [12]:

# Sample financial transaction amounts ($)
# Includes typical amounts and a few potential outliers (very high or low)
transaction_amounts = np.array([
    55.20, 68.15, 75.00, 88.90, 105.50, 62.30, 79.80, 95.10,
    110.00, 125.60, 4500.00, # <-- Likely high outlier
    72.50, 85.40, 99.99, 108.20, 5.50,    # <-- Likely low outlier
    91.75, 115.30, 66.80, 81.00, 103.40, 5200.50, # <-- Another high outlier
    78.60, 93.25
])

print("Original Transaction Amounts:")
print(transaction_amounts)
print("-" * 40)

# Set the threshold for outlier detection
# A threshold of 3.5 is often used, meaning points more than 3.5
# median absolute deviations away from the median are flagged.
mad_threshold = 3.5
print(f"Using MAD threshold: {mad_threshold}")
print("-" * 40)

# Detect outliers using the function
identified_outliers, data_median, data_mad, mod_z_scores = detect_outliers_mad(
    transaction_amounts,
    threshold=mad_threshold
)

print(f"Data Median: ${data_median:.2f}")
print(f"Data MAD: ${data_mad:.2f}")
print(identified_outliers)

Original Transaction Amounts:
[  55.2    68.15   75.     88.9   105.5    62.3    79.8    95.1   110.
  125.6  4500.     72.5    85.4    99.99  108.2     5.5    91.75  115.3
   66.8    81.    103.4  5200.5    78.6    93.25]
----------------------------------------
Using MAD threshold: 3.5
----------------------------------------
Data Median: $90.33
Data MAD: $16.58
[4500.  5200.5]
